# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [10]:
def square_root_generator(limit):
    n = 1
    x = 0
    while n <= limit:
        yield n ** 0.5
        x = x+n**0.5
        print(f'{n}th Sum:{x}')
        n += 1

# Example usage:
limit = 13
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)


1.0
1th Sum:1.0
1.4142135623730951
2th Sum:2.414213562373095
1.7320508075688772
3th Sum:4.146264369941973
2.0
4th Sum:6.146264369941973
2.23606797749979
5th Sum:8.382332347441762
2.449489742783178
6th Sum:10.83182209022494
2.6457513110645907
7th Sum:13.47757340128953
2.8284271247461903
8th Sum:16.30600052603572
3.0
9th Sum:19.30600052603572
3.1622776601683795
10th Sum:22.4682781862041
3.3166247903554
11th Sum:25.7849029765595
3.4641016151377544
12th Sum:29.249004591697254
3.605551275463989
13th Sum:32.854555867161245


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [12]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

# for person in people_1():
#     print(person)



def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


# for person in people_2():
#     print(person)


{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [25]:
import dlt

# define the connection to load to.
# We now use duckdb, but you can switch to Bigquery later
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='dlt_hw')

info = generators_pipeline.run(people_1(),
                              table_name = "people_1",
                              write_disposition = "replace"
                              )

info = generators_pipeline.run(people_2(),
                              table_name = "people_1",
                              write_disposition = "append"
                              )


In [26]:
import duckdb
# print((f"{generators_pipeline.pipeline_name}.duckdb"))
# print(generators_pipeline.dataset_name)
conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
# print('Loaded Tables: ')
# display(conn.sql("show tables"))

hw_q_3 = conn.sql("select sum(age) from people_1").df()
display(hw_q_3)

,sum(age)
0,353.0


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


In [40]:


# define the connection to load to.
# We now use duckdb, but you can switch to Bigquery later
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='dlt_hw')

info = generators_pipeline.run(people_1(),
                              table_name = "people_merged",
                              write_disposition = "replace",
                              primary_key="ID"
                              )

info = generators_pipeline.run(people_2(),
                              table_name = "people_merged",
                              write_disposition = "merge",
                              primary_key="ID"
                              )

In [42]:

# print((f"{generators_pipeline.pipeline_name}.duckdb"))
# print(generators_pipeline.dataset_name)
conn1 = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")
conn1.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
# print('Loaded Tables: ')
# display(conn.sql("show tables"))

hw_q_4 = conn1.sql("select sum(age) from people_merged").df()
display(hw_q_4)

,sum(age)
0,266.0


# Solution: First make sure that the following modules are installed:

In [ ]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [ ]:
# to do: homework :)

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX